# Water Mass Transformation diagnostics using ACCESS-OM2 neutral diagnostics

This is a example code for working with the Water Mass Transformation output from the neutral `_on_nrho` diagnostics for a tracer

In [7]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import os

import IPython.display

import cosima_cookbook as cc

At the moment the diagnostics have to be output from a particular run or runs of ACCESS-OM2. Specify the output directory on Gadi

In [4]:
output_dir = r'/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/'

In [25]:
dir_list = sorted([os.path.join(output_dir, _, 'ocean') for _ in os.listdir(output_dir) if _.startswith('output')])
dir_list

['/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output050/ocean',
 '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output051/ocean',
 '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output052/ocean',
 '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output053/ocean',
 '/scratch/e14/cb2411/access-om2/archive/1deg_jra55_ryf/output054/ocean']

To make the database, specify a location you have write access to (assign `db_path`)

In [22]:
db_path = r'/g/data/e14/cb2411/data'
session = cc.database.create_session(os.path.join(db_path, 'wmt_diags.db'))

In [26]:
cc.database.build_index(dir_list,session,update=True)

  0%|          | 0/5 [00:00<?, ?it/s]

Indexing experiment: ocean


  0%|          | 0/8 [00:00<?, ?it/s]

Indexing experiment: ocean


 12%|█▎        | 1/8 [00:00<00:00,  8.45it/s]

...

Indexing experiment: ocean


 25%|██▌       | 2/8 [00:00<00:00, 11.30it/s]

Indexing experiment: ocean


100%|██████████| 8/8 [00:00<00:00, 11.35it/s]


37